<a href="https://colab.research.google.com/github/subdiduntil2/AVLSI_ECE_NTUA/blob/main/training_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install torch-geometric
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 9.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=61fbf93a9c48a0b3d658fffdbd23b73a06ad6e1f9a14111fb3b860f915c64d39
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric


In [10]:
#models and pipeline dependencies
import os
import warnings
import numpy as np
import torch
import math
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import Set2Set, AttentionalAggregation, MeanAggregation, GATConv
from torch_geometric.nn.aggr import Aggregation
from torch_geometric.nn.inits import reset
from torch_geometric.utils import softmax
from typing import Optional
from torch import Tensor
# from models import *
from torchsummary import summary
from prettytable import PrettyTable
from typing import Tuple
import torch.nn as nn
from torch.nn.functional import softmax
import torch_geometric
from torch.functional import F
import numpy as np
from google.colab import drive
from math import log2
drive.mount('/content/drive')
print("libraries loaded")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
libraries loaded


In [27]:
#functions and outside params

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

num_node_feats = 11
num_edge_feats = 2
num_edges = 6000
num_nodes = 3000
num_graphs = 20
num_embs = 1000 #tbc
emb_dim = 100
batch_size = 1
epochs = 6
loss_all = 0.0
max_split_size_mb = 200
data_list = []
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device is -> ",DEVICE)
print("loaded params and functions")

device is ->  cuda
loaded params and functions


In [28]:
#model class definitions
class AttentionalAggregationCustom(Aggregation):
    def __init__(self, gate_nn: torch.nn.Module,nn: Optional[torch.nn.Module] = None):
        super().__init__()
        self.gate_nn = gate_nn
        self.nn = nn
        # self.reset_parameters()

    def reset_parameters(self):
        reset(self.gate_nn)
        reset(self.nn)

    def forward(self, x: Tensor, index: Optional[Tensor] = None, ptr: Optional[Tensor] = None, dim_size: Optional[int] = None, dim: int = -2) -> Tensor:
        self.assert_two_dimensional_input(x, dim)
        gate = self.gate_nn(x)
        x = self.nn(x) if self.nn is not None else x
        print("gate gives -> ",gate,type(gate))
        print("x is -> ",x,np.shape(x))
        gate = softmax(gate, index, ptr, dim_size, dim)
        return self.reduce(gate * x, index, ptr, dim_size, dim)

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}(gate_nn={self.gate_nn}, '
                f'nn={self.nn})')

class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    def __init__(self, in_channels, dropout=0.0):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_channels, 4*in_channels),
            nn.ReLU(),
            nn.Linear(4*in_channels, in_channels),
            nn.Dropout(dropout),
        )
        self.init_weights(FeedForward)

    def init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight)
            nn.init.zeros_(m.bias)

    def forward(self, x):
        return self.net(x)

class GraphAttentionLayer(nn.Module):
    def __init__(self, in_features, out_features, n_heads=3 , concat = True, dropout = 0.1, leaky_relu_slope = 0.1):
        super(GraphAttentionLayer, self).__init__()

        self.n_heads = n_heads # Number of attention heads
        self.concat = concat # wether to concatenate the final attention heads
        self.dropout = dropout # Dropout rate

        # if concat: # concatenating the attention heads
        #     self.out_features = out_features # Number of output features per node
        #     assert out_features % n_heads == 0 # Ensure that out_features is a multiple of n_heads
        #     self.n_hidden = out_features // n_heads
        # else: # averaging output over the attention heads (Used in the main paper)
        #     self.n_hidden = out_features
        self.out_features = out_features
        self.n_hidden = out_features

        #  A shared linear transformation, parametrized by a weight matrix W is applied to every node
        #  Initialize the weight matrix W
        self.W = nn.Parameter(torch.empty(size=(in_features, self.n_hidden * n_heads)))

        # Initialize the attention weights a
        self.a = nn.Parameter(torch.empty(size=(n_heads, 2 * self.n_hidden, 1)))

        self.leakyrelu = nn.LeakyReLU(leaky_relu_slope) # LeakyReLU activation function
        self.softmax = nn.Softmax(dim=1) # softmax activation function to the attention coefficients

        self.reset_parameters() # Reset the parameters


    def reset_parameters(self):
        """
        Reinitialize learnable parameters.
        """
        nn.init.xavier_normal_(self.W)
        nn.init.xavier_normal_(self.a)


    def _get_attention_scores(self, h_transformed: torch.Tensor):

        source_scores = torch.matmul(h_transformed, self.a[:, :self.n_hidden, :])
        target_scores = torch.matmul(h_transformed, self.a[:, self.n_hidden:, :])

        # broadcast add
        # (n_heads, n_nodes, 1) + (n_heads, 1, n_nodes) = (n_heads, n_nodes, n_nodes)
        e = source_scores + target_scores.mT
        return self.leakyrelu(e)

    def forward(self,  h: torch.Tensor, adj_mat: torch.Tensor):
        n_nodes = h.shape[0]

        # Apply linear transformation to node feature -> W h
        # output shape (n_nodes, n_hidden * n_heads)
        h_transformed = torch.mm(h, self.W)
        h_transformed = F.dropout(h_transformed, self.dropout, training=self.training)

        # splitting the heads by reshaping the tensor and putting heads dim first
        # output shape (n_heads, n_nodes, n_hidden)
        h_transformed = h_transformed.view(n_nodes, self.n_heads, self.n_hidden).permute(1, 0, 2)

        # getting the attention scores
        # output shape (n_heads, n_nodes, n_nodes)
        e = self._get_attention_scores(h_transformed)

        # Set the attention score for non-existent edges to -9e15 (MASKING NON-EXISTENT EDGES)
        connectivity_mask = -9e16 * torch.ones_like(e)
        adj_mat = adj_mat.to(device)
        print("adj_matrix shape is -> ",np.shape(adj_mat))
        e = torch.where(adj_mat > 0, e, connectivity_mask) # masked attention scores

        # attention coefficients are computed as a softmax over the rows
        # for each column j in the attention score matrix e
        attention = F.softmax(e, dim=-1)
        attention = F.dropout(attention, self.dropout, training=self.training)

        # final node embeddings are computed as a weighted average of the features of its neighbors
        h_prime = torch.matmul(attention, h_transformed)

        # concatenating/averaging the attention heads
        # output shape (n_nodes, out_features)
        h_prime = h_prime.mean(dim=0)

        return h_prime

class MultiHeadGraphAttention(nn.Module):
    """ multiple heads of graph self-attention in parallel """
    #documentation for GATConv class -> https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.GATConv.html
    def __init__(self, num_feats, h_channels=8, num_heads=3, dropout=0.1, alpha=0.2, masked=False):
        super().__init__()
        # self.conv = [GraphAttentionLayer(num_feats, h_channels, n_heads=num_heads, dropout=dropout, leaky_relu_slope=alpha) for _ in range(num_heads)]
        # self.conv = GATConv(num_feats, h_channels, heads=num_heads, dropout=dropout, negative_slope=alpha)
        self.conv = GraphAttentionLayer(num_feats, num_heads*h_channels, n_heads=num_heads, dropout=dropout, leaky_relu_slope=alpha)
        self.proj = nn.Linear(h_channels*num_heads, num_feats)
        self.dropout = nn.Dropout(dropout)
    # def __init__(self, in_features, out_features, n_heads=3 , concat = True, dropout = 0.1, leaky_relu_slope = 0.1):
    def forward(self, x, edge_index, edge_attr):
        # print("x is -> ",x, np.shape(x),type(x))
        # print("edge idx is -> ",edge_index, np.shape(edge_index),type(edge_index))
        adj_matrix = self.return_adj(edge_index)
        # head_outputs = self.conv(x,edge_index,edge_attr) # List of head outputs
        # head_outputs = torch.cat([att(x, adj_matrix) for att in self.conv], dim=1)
        head_outputs = self.conv(x,adj_matrix)
        print("head_outputs is ",head_outputs,np.shape(head_outputs),type(head_outputs))
        out = self.dropout(head_outputs)
        out = self.proj(out)
        return out

    def return_adj(self,edge_index):
      global num_nodes
      num_nodes = edge_index.max().item() + 1
      # print("num_nodes local is -> ",num_nodes)
      adjacency_matrix = torch.zeros((num_nodes, num_nodes), dtype=torch.float)
      adjacency_matrix[edge_index[:, 0], edge_index[:, 1]] = 1
      return adjacency_matrix

class GraphBlock(nn.Module):
    """ Transformer block: communication followed by computation """
    def __init__(self, num_feats, num_heads, h_channels, masked):
        # num_feats: feature space dimension, n_head: the number of heads we'd like
        super().__init__()
        self.sa = MultiHeadGraphAttention(num_feats,num_heads=num_heads,h_channels=h_channels, masked=masked)
        # print("number of feats in feedforward is -> ",num_feats)
        self.ffwd = FeedForward(num_feats)
        self.ln1 = nn.LayerNorm(num_feats)
        self.ln2 = nn.LayerNorm(num_feats)

    def forward(self, x, edge_index, edge_attr):
        x = self.ln1(x)
        # print("x_gb after layer_norm is -> ",x, np.shape(x),type(x))
        x = self.sa(x,edge_index,edge_attr)
        # print("x_gb after mhga is -> ",x, np.shape(x),type(x))
        x = self.ln2(x)
        # print("x_gb after layer_norm is -> ",x, np.shape(x),type(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class PriorDiscriminator(torch.nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.l0 = torch.nn.Linear(input_dim, input_dim)
        self.l1 = torch.nn.Linear(input_dim, input_dim)
        self.l2 = torch.nn.Linear(input_dim, 1)

    def forward(self, x):
        h = F.relu(self.l0(x))
        h = F.relu(self.l1(h))
        return torch.sigmoid(self.l2(h))

class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.2, max_len: int = 300):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        # print("div_term is -> ",div_term, np.shape(div_term),type(div_term))
        # print("pe is -> ",pe, np.shape(pe),type(pe))
        # print("pe1 is -> ",pe[:, 0, 0::2],np.shape(pe[:, 0, 0::2]),type(pe[:, 0, 0::2]))
        # print("pe2 is -> ",pe[:, 0, 1::3],np.shape(pe[:, 0, 1::3]),type(pe[:, 0, 1::3]))
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 0::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self,x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

class GraphTransformerEncoderModel(nn.Module):
    def __init__(self, num_feats, num_embs, emb_dim, output_size, max_length, h_channels=8, num_heads=3, n_layers=2, decoder_bool=False, masked=False, train_embs = False):
        super().__init__()
        self.num_heads = num_heads
        self.decoder_bool = decoder_bool
        self.output_size = output_size
        self.emb_layer = nn.Embedding(num_embs,emb_dim,sparse=True)
        self.emb_layer.weight.requires_grad = train_embs
        # self.pos_emb
        if train_embs: self.init_weights(self.emb_layer)
        self.pos_enc = PositionalEncoding(emb_dim,max_len=max_length)
        self.blocks = nn.Sequential(*[GraphBlock(emb_dim*num_feats, num_heads, h_channels=h_channels, masked=masked) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(num_feats*emb_dim)  # final layer norm
        self.output = nn.Linear(num_feats*emb_dim, output_size)
        self.emb_act = nn.ReLU()

    def forward(self, x, edge_index, edge_attr):
        # x = self.emb_layer(x)
        print("x before emb is -> ",x,np.shape(x))
        x1 = self.emb_layer(x)
        x2 = self.pos_enc(x1)
        x = x1+x2
        # print("x after emb is ",np.shape(x),np.shape(x1),np.shape(x2))
        if(np.shape(x.shape)[0]==3):
            B,C,D = x.shape
            x = x.view(B, C*D)
        print("x after emb. is -> ",np.shape(x))
        for block in self.blocks: x = block(x, edge_index, edge_attr)
        x = self.ln_f(x)
        x = self.output(x)
        if(self.decoder_bool==False):
            x_relu = self.emb_act(x)
            return x_relu
        else: return x

class FinalModel(nn.Module):
    def __init__(self, num_feats, num_embs, emb_dim, output_size, max_length, processing_steps=3, beta=1.0):
        super().__init__()
        self.output_size = output_size
        self.output_size_enc = output_size
        self.output_size_disc = 1
        self.emb_dim = emb_dim
        self.num_nodes = num_nodes
        self.beta = beta
        self.encoder = GraphTransformerEncoderModel(num_feats,num_embs,emb_dim,self.output_size_enc,max_length)
        self.aggr_oper = Set2Set(self.output_size_enc,processing_steps)
        # gate_custom = nn.LSTM(self.output_size_enc,8)
        # self.aggr_oper = AttentionalAggregationCustom(gate_custom)
        # self.aggr_oper = MeanAggregation()
        self.local_d = Discriminator(self.output_size_enc,self.output_size_disc)
        self.global_d = Discriminator(2*self.output_size_enc,self.output_size_disc)
        self.prior_d = PriorDiscriminator(self.output_size_disc)
        self.mutual_l = MutualInformation()
        self.cosine_l = nn.CosineSimilarity(dim=1, eps=1e-6)
        self.prop_l = nn.BCELoss()
        self.init_emb()
        # self.cosine_l = nn.CosineEmbeddingLoss()

    def forward(self, x, edge_index, edge_attr):
        x_enc = self.encoder(x,edge_index,edge_attr)
        B,C = x_enc.shape
        print("shapes are -> ",B," ",C)
        x_set = self.aggr_oper(x_enc,dim_size=1)
        print("aggregator gives -> ",x_set,np.shape(x_set))
        print("encoder gives -> ",x_enc,np.shape(x_enc))
        x_enc_d = self.local_d(x_enc)
        x_set_d = self.global_d(x_set)
        # x_enc_d = torch.matmul(x_enc_d,x_set_d.t())
        return x_enc_d, x_set_d

    def mutual_loss(self, arr1, arr2):
        return self.mutual_l(arr1,arr2)

    def cosine_loss(self,arr1,arr2):
        return self.cosine_l(arr1,arr2)

    def probability_loss(self,arr1,arr2):
        return self.prop_l(arr1,arr2)

    def local_global_loss_(self,l_enc, g_enc, edge_index, batch):
      num_nodes = l_enc.shape[0]
      num_graphs = g_enc.shape[0]
      print("num of nodes and num of graphs are -> ", num_nodes, " ",num_graphs)

      pos_mask = torch.zeros((num_nodes, num_graphs)).cuda()
      neg_mask = torch.ones((num_nodes, num_graphs)).cuda()
      for nodeidx, graphidx in enumerate(batch):
          pos_mask[nodeidx][graphidx] = 1
          neg_mask[nodeidx][graphidx] = 0

      res = torch.mm(l_enc, g_enc.t())
      log_2 = math.log(2.)

      p_samples = res*pos_mask
      E_pos = log_2 - F.softplus(- p_samples)
      E_pos = E_pos.sum()
      E_pos = E_pos/self.num_nodes

      n_samples = res*neg_mask
      E_neg = F.softplus(-n_samples) + n_samples - log_2
      E_neg = E_neg.sum()
      E_neg = E_neg / (num_nodes * (num_graphs - 1))

      prior = torch.rand_like(g_enc)

      term_a = torch.log(self.prior_d(prior)).mean()
      term_b = torch.log(1.0 - self.prior_d(g_enc)).mean()
      PRIOR = -(term_a + term_b) * self.beta

      return (E_neg - E_pos) + PRIOR

    def init_emb(self):
      initrange = -1.5 / self.emb_dim
      for m in self.modules():
          if isinstance(m, nn.Linear):
              torch.nn.init.xavier_uniform_(m.weight.data)
              if m.bias is not None: m.bias.data.fill_(0.0)

class Discriminator(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.disc_block = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 2*input_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(2*input_dim, 4*input_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(4*input_dim, output_dim),
            torch.nn.ReLU()
        )
        self.soft_ = torch.nn.Softmax()
        self.sigm_ = nn.Sigmoid()
        self.linear_shortcut = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x1 = self.disc_block(x)
        x2 = self.linear_shortcut(x)
        xol = x1+x2
        return self.sigm_(xol)

class MutualInformation(nn.Module):
	def __init__(self, sigma=0.1, num_bins=256, normalize=True):
		super(MutualInformation, self).__init__()
		self.sigma = sigma
		self.num_bins = num_bins
		self.normalize = normalize
		self.epsilon = 1e-10
		self.bins = nn.Parameter(torch.linspace(0, 255, num_bins).float(), requires_grad=False)

	def marginalPdf(self, values):
		residuals = values - self.bins.unsqueeze(0).unsqueeze(0)
		kernel_values = torch.exp(-0.5*(residuals / self.sigma).pow(2))
		pdf = torch.mean(kernel_values, dim=1)
		normalization = torch.sum(pdf, dim=1).unsqueeze(1) + self.epsilon
		pdf = pdf / normalization
		return pdf, kernel_values

	def jointPdf(self, kernel_values1, kernel_values2):
		joint_kernel_values = torch.matmul(kernel_values1.transpose(1, 2), kernel_values2)
		normalization = torch.sum(joint_kernel_values, dim=(1,2)).view(-1, 1, 1) + self.epsilon
		pdf = joint_kernel_values / normalization
		return pdf

	def getMutualInformation(self, input1, input2):
		# Torch tensors for images between (0, 1)
		input1 = input1*256
		input2 = input2*256
		B, C = input1.shape
		assert((input1.shape == input2.shape))
		x1 = input1.view(B, C, 1)
		x2 = input2.view(B, C, 1)
		pdf_x1, kernel_values1 = self.marginalPdf(x1)
		pdf_x2, kernel_values2 = self.marginalPdf(x2)
		pdf_x1x2 = self.jointPdf(kernel_values1, kernel_values2)
		H_x1 = -torch.sum(pdf_x1*torch.log2(pdf_x1 + self.epsilon), dim=1)
		H_x2 = -torch.sum(pdf_x2*torch.log2(pdf_x2 + self.epsilon), dim=1)
		H_x1x2 = -torch.sum(pdf_x1x2*torch.log2(pdf_x1x2 + self.epsilon), dim=(1,2))
		mutual_information = H_x1 + H_x2 - H_x1x2
		if self.normalize:
			mutual_information = 2*mutual_information/(H_x1+H_x2)
		return mutual_information

	def forward(self, input1, input2):
		return self.getMutualInformation(input1, input2)

print("loading of models done")

loading of models done


In [29]:
#data generation and loading
for i in range(num_graphs): #create dummy data
    edge_index = torch.randint(1,num_nodes,(2,num_edges))
    x = torch.mul(torch.rand(num_nodes,num_node_feats),100).type(torch.int64)
    print("x bef is -> ",x, np.shape(x),type(x))
    edge_attr = torch.rand(num_edges,num_edge_feats)
    data_temp = Data(x=x, edge_index=edge_index,edge_attr=edge_attr)
    data_list.append(data_temp)
print("data list is -> ",data_list,np.shape(data_list),type(data_list))
dataloader = DataLoader(data_list, batch_size=batch_size, shuffle=True)
print("dataloader is -> ",dataloader,np.shape(dataloader),type(dataloader))
print()
# DEVICE = 'cpu'
model = FinalModel(num_feats=num_node_feats, num_embs=num_embs, emb_dim=emb_dim, output_size=num_node_feats, max_length=int(batch_size*num_nodes))
torch.cuda.empty_cache()
print("pytorch version is: ",torch.__version__)
# set_max_split_size_mb(model, dummy_x, dummy_edge_index, dummy_edge_attr, max_split_size_mb)
# print(summary(model,[(num_nodes,num_node_feats),(2,num_edges),(num_edges,num_edge_feats)]))
print("model gives -> ",count_parameters(model))
print("device is -> ",DEVICE, torch.cuda.is_available())
model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) # Initialize the Adam optimizer.
# optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9) #Initialize the SDG optimizer
for index,batch in enumerate(dataloader):
    print("inside index i have",index,batch,np.shape(batch),batch.num_graphs)
    print("inside index_2 i have",index,batch.x,np.shape(batch.x),type(batch.x))
print("dataloading finished")

x bef is ->  tensor([[49, 45, 47,  ..., 55, 12, 94],
        [72, 30, 86,  ..., 55, 96, 33],
        [59, 17, 93,  ..., 65, 58, 29],
        ...,
        [55, 29, 27,  ..., 29, 19, 29],
        [14, 77, 70,  ..., 89, 10, 27],
        [54,  4, 20,  ..., 30, 93, 88]]) torch.Size([3000, 11]) <class 'torch.Tensor'>
x bef is ->  tensor([[69, 65, 15,  ..., 17, 69, 10],
        [99, 38, 25,  ..., 64, 27, 96],
        [64, 79, 86,  ..., 35, 49, 69],
        ...,
        [62, 28, 96,  ..., 32, 95, 16],
        [42, 80, 40,  ..., 11, 77, 54],
        [23,  9, 55,  ..., 71, 86, 55]]) torch.Size([3000, 11]) <class 'torch.Tensor'>
x bef is ->  tensor([[99, 95, 40,  ..., 10,  8, 84],
        [87, 57, 58,  ...,  6, 68,  7],
        [10, 40, 38,  ..., 55, 70, 24],
        ...,
        [55, 10, 52,  ..., 92, 52, 68],
        [11, 63, 35,  ..., 55, 22,  2],
        [45, 32, 86,  ..., 75, 33, 64]]) torch.Size([3000, 11]) <class 'torch.Tensor'>
x bef is ->  tensor([[49, 48,  3,  ..., 25, 21, 73],
        

In [30]:
!nvidia-smi

Mon Sep 11 07:25:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    29W /  70W |   3865MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [32]:
#main training loop
log_interval = 5
for epoch in range(1, epochs+1):
    loss_all = 0
    device = next(model.parameters()).device
    model.train()
    for index,data in enumerate(dataloader):
      data = data.to(device)
      optimizer.zero_grad()
      print("index of dataloader is" , index)
      print("node feats are",data.x,np.shape(data.x),type(data.x))
      local_d, global_d = model(data.x, data.edge_index, data.edge_attr)
      global_d = global_d.item()*torch.ones(num_nodes,1).to(device)
      discr_global = torch.unique(global_d).numel()
      print("local_output",local_d,np.shape(local_d),type(local_d))
      print("global_output",global_d,np.shape(global_d),type(global_d),discr_global)
      loss = model.local_global_loss_(local_d,global_d,data.edge_index,data.batch)
      # loss = model.probability_loss(local_d,global_d)
      loss_batch = loss
      print("loss_init is: ",loss,np.shape(loss),type(loss))
      print("loss is ", loss," ",np.shape(loss)," ",type(loss))
      loss_final = torch.tensor([loss_batch.item()],requires_grad=True)
      loss_all += loss_final*num_graphs
      print("loss final ", loss_final,np.shape(loss_final),type(loss_final))
      loss_final.backward()
      optimizer.step()
    print('===== Epoch {}, Loss {} ====='.format(epoch, loss_all / len(dataloader)))
    if epoch % log_interval == 0:
      print("mpika evaluate mode!!")
      model.eval()
      data = data.to(device)
      local_d, global_d = model(data.x,data.edge_index,data.edge_attr)
      print("local_scores are -> ",local_d," ",np.shape(local_d)," at epoch: ",epoch)
      print("global_scores are -> ",global_d," ",np.shape(global_d)," at epoch: ",epoch)
torch.save(model.state_dict(), '/content/drive/My Drive/model.pth')

Streaming output truncated to the last 5000 lines.
       device='cuda:0', grad_fn=<MeanBackward1>) torch.Size([3000, 24]) <class 'torch.Tensor'>
shapes are ->  3000   11
aggregator gives ->  tensor([[ 0.3945,  0.1231, -0.0152, -0.2676, -0.0490,  0.0803, -0.0742,  0.1612,
          0.1488,  0.3828, -0.0265,  0.0050,  1.9961,  0.1451,  1.9370,  0.0399,
          0.0637,  1.0154,  1.3854,  0.0000,  1.9509,  0.1704]],
       device='cuda:0', grad_fn=<CatBackward0>) torch.Size([1, 22])
encoder gives ->  tensor([[0.0000, 1.7089, 0.0000,  ..., 0.0000, 1.4488, 0.2043],
        [0.0000, 1.5528, 0.0000,  ..., 0.0000, 2.2413, 0.3035],
        [0.0000, 1.8198, 0.0000,  ..., 0.0000, 1.7096, 0.2486],
        ...,
        [0.0000, 2.1291, 0.0000,  ..., 0.0000, 0.4621, 0.0893],
        [0.0000, 0.8741, 0.0000,  ..., 0.0000, 1.9433, 0.0000],
        [0.0000, 1.5633, 0.1545,  ..., 0.0000, 1.4337, 0.0393]],
       device='cuda:0', grad_fn=<ReluBackward0>) torch.Size([3000, 11])
local_output tensor([[0.6

In [ ]:
with torch.no_grad():
    torch.cuda.empty_cache()